<a href="https://colab.research.google.com/github/LeHoangNhuan/ROBOT_H-N/blob/main/GOCAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import requests
import json
from datetime import datetime, timedelta
import pytz
import time

# Khởi tạo geocoder với user agent tùy chỉnh
geolocator = Nominatim(user_agent="grab_simulator_v2.0")

# Hàm tìm kiếm địa chỉ với nhiều gợi ý
def search_address_suggestions(query, limit=8):
    """
    Tìm kiếm địa chỉ và trả về nhiều gợi ý như Google Maps
    """
    try:
        # Tìm kiếm với từ khóa chính
        search_queries = [
            f"{query}, Ho Chi Minh City, Vietnam",
            f"{query}, Saigon, Vietnam",
            f"{query}, HCMC, Vietnam",
            f"{query}, Vietnam"
        ]

        suggestions = []
        seen_coords = set()

        for search_query in search_queries:
            try:
                locations = geolocator.geocode(search_query, exactly_one=False, limit=limit, timeout=10)

                if locations:
                    for location in locations:
                        # Tránh trùng lặp dựa trên tọa độ
                        coord_key = (round(location.latitude, 4), round(location.longitude, 4))
                        if coord_key not in seen_coords:
                            seen_coords.add(coord_key)

                            # Rút gọn tên địa chỉ cho dễ đọc
                            display_name = location.address
                            if len(display_name) > 80:
                                parts = display_name.split(',')
                                if len(parts) > 3:
                                    display_name = ', '.join(parts[:3]) + '...'

                            suggestions.append({
                                'coordinates': (location.latitude, location.longitude),
                                'display_name': display_name,
                                'short_name': parts[0] if ',' in display_name else display_name[:50]
                            })

                            if len(suggestions) >= limit:
                                break

            except Exception as e:
                continue

            if len(suggestions) >= limit:
                break

        return {
            'suggestions': suggestions,
            'success': len(suggestions) > 0,
            'error': None if len(suggestions) > 0 else 'Không tìm thấy địa chỉ phù hợp'
        }

    except Exception as e:
        return {
            'suggestions': [],
            'success': False,
            'error': str(e)
        }

def get_current_datetime():
    """Lấy ngày giờ hiện tại theo múi giờ Việt Nam"""
    try:
        vietnam_tz = pytz.timezone('Asia/Ho_Chi_Minh')
        current_time = datetime.now(vietnam_tz)

        formatted_time = current_time.strftime("%H:%M:%S")
        formatted_date = current_time.strftime("%d/%m/%Y")
        day_of_week = current_time.strftime("%A")

        days_vietnamese = {
            "Monday": "Thứ Hai", "Tuesday": "Thứ Ba", "Wednesday": "Thứ Tư",
            "Thursday": "Thứ Năm", "Friday": "Thứ Sáu", "Saturday": "Thứ Bảy",
            "Sunday": "Chủ Nhật"
        }

        vietnamese_day = days_vietnamese.get(day_of_week, day_of_week)

        return {
            'time': formatted_time,
            'date': formatted_date,
            'day': vietnamese_day,
            'datetime_obj': current_time
        }
    except:
        current_time = datetime.now()
        return {
            'time': current_time.strftime("%H:%M:%S"),
            'date': current_time.strftime("%d/%m/%Y"),
            'day': "N/A",
            'datetime_obj': current_time
        }

def get_osrm_route(start_coords, end_coords, profile='driving'):
    """Lấy tuyến đường từ OSRM"""
    try:
        coords_str = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
        url = f"http://router.project-osrm.org/route/v1/{profile}/{coords_str}?overview=full&geometries=geojson"

        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            data = response.json()
            if data['code'] == 'Ok' and data['routes']:
                route = data['routes'][0]
                coordinates = route['geometry']['coordinates']
                route_coords = [[coord[1], coord[0]] for coord in coordinates]
                distance = route['distance']
                duration = route['duration']
                return route_coords, distance, duration

        return None, None, None

    except Exception as e:
        print(f"Lỗi OSRM: {e}")
        return None, None, None

def calculate_fare_12k_per_km(distance_km, base_fare=8000, min_fare=15000):
    """
    Tính toán giá cước với rate 12000 VND/km, bao gồm giá khởi điểm và giá tối thiểu.

    Args:
        distance_km (float): Khoảng cách tính bằng km.
        base_fare (int): Giá khởi điểm.
        min_fare (int): Giá tối thiểu.

    Returns:
        int: Tổng cước phí.
    """
    distance_fare = distance_km * 12000
    total_fare = base_fare + distance_fare

    if total_fare < min_fare:
        total_fare = min_fare

    return int(total_fare)


# Tạo giao diện
current_datetime = get_current_datetime()

# Header
header = widgets.HTML(f"""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; padding: 20px; border-radius: 15px; text-align: center; margin-bottom: 15px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);'>
    <h2 style='margin: 0; font-size: 28px; font-weight: 300;'>🚖 GOCAR - Xe Của Mọi Nhà</h2>
    <div style='margin-top: 10px; font-size: 16px; opacity: 0.9;'>
        <span style='margin-right: 25px;'>📅 {current_datetime['day']}, {current_datetime['date']}</span>
        <span>🕐 {current_datetime['time']}</span>
    </div>
    <div style='margin-top: 8px; font-size: 14px; opacity: 0.8;'>
        Tìm kiếm thông minh - Gợi ý tự động
    </div>
</div>
""")

# Tạo các widget tìm kiếm
start_input = widgets.Text(
    placeholder='🔍 Nhập địa chỉ điểm đi (VD: Chợ Bến Thành, 123 Nguyễn Huệ...)',
    layout=widgets.Layout(width='100%', height='45px'),
    style={'description_width': '0px'}
)

start_search_btn = widgets.Button(
    description='🔍 TÌM KIẾM',
    button_style='info',
    layout=widgets.Layout(width='120px', height='45px')
)

start_suggestions = widgets.HTML()
start_selected = widgets.HTML()

end_input = widgets.Text(
    placeholder='🎯 Nhập địa chỉ điểm đến (VD: Sân bay Tân Sơn Nhất...)',
    layout=widgets.Layout(width='100%', height='45px'),
    style={'description_width': '0px'}
)

end_search_btn = widgets.Button(
    description='🔍 TÌM KIẾM',
    button_style='info',
    layout=widgets.Layout(width='120px', height='45px')
)

end_suggestions = widgets.HTML()
end_selected = widgets.HTML()

# Biến lưu trữ
selected_start = None
selected_end = None
start_suggestions_data = []
end_suggestions_data = []

# Hàm tạo buttons cho gợi ý
def create_suggestion_buttons(suggestions_data, is_start=True):
    """Tạo buttons cho từng gợi ý để có thể click được"""
    if not suggestions_data:
        return []

    buttons = []
    for i, suggestion in enumerate(suggestions_data):
        btn = widgets.Button(
            description=f"📍 {suggestion['short_name'][:40]}{'...' if len(suggestion['short_name']) > 40 else ''}",
            layout=widgets.Layout(width='100%', margin='2px'),
            style={'button_color': '#f8f9fa', 'text_color': '#333'},
            tooltip=suggestion['display_name']
        )

        # Tạo callback function
        if is_start:
            btn.on_click(lambda b, idx=i: select_start_suggestion(idx))
        else:
            btn.on_click(lambda b, idx=i: select_end_suggestion(idx))

        buttons.append(btn)

    return buttons

# Container để chứa suggestion buttons
start_suggestions_container = widgets.VBox([])
end_suggestions_container = widgets.VBox([])

# Hàm xử lý tìm kiếm điểm đi
def search_start_suggestions(b):
    global start_suggestions_data, selected_start, start_suggestions_container

    query = start_input.value.strip()
    if not query:
        start_suggestions.value = "<div style='color: #ff6b6b; padding: 10px;'>⚠️ Vui lòng nhập địa chỉ</div>"
        start_suggestions_container.children = []
        return

    if len(query) < 3:
        start_suggestions.value = "<div style='color: #ffa500; padding: 10px;'>⚠️ Nhập ít nhất 3 ký tự</div>"
        start_suggestions_container.children = []
        return

    start_suggestions.value = "<div style='color: #4ecdc4; padding: 15px; text-align: center;'>🔍 Đang tìm kiếm gợi ý...</div>"
    start_suggestions_container.children = []

    result = search_address_suggestions(query, limit=6)

    if result['success'] and result['suggestions']:
        start_suggestions_data = result['suggestions']

        start_suggestions.value = f"""
        <div style='color: #4ecdc4; font-weight: 500; margin-bottom: 8px; padding: 0 5px;'>
            ✅ Tìm thấy {len(start_suggestions_data)} gợi ý - Nhấn để chọn:
        </div>
        """

        # Tạo buttons cho gợi ý
        suggestion_buttons = create_suggestion_buttons(start_suggestions_data, True)
        start_suggestions_container.children = tuple(suggestion_buttons) # Assign a tuple of buttons

    else:
        start_suggestions_data = []
        start_suggestions.value = f"<div style='color: #ff6b6b; padding: 10px;'>❌ {result['error']}</div>"
        start_suggestions_container.children = []

# Hàm xử lý tìm kiếm điểm đến
def search_end_suggestions(b):
    global end_suggestions_data, selected_end, end_suggestions_container

    query = end_input.value.strip()
    if not query:
        end_suggestions.value = "<div style='color: #ff6b6b; padding: 10px;'>⚠️ Vui lòng nhập địa chỉ</div>"
        end_suggestions_container.children = []
        return

    if len(query) < 3:
        end_suggestions.value = "<div style='color: #ffa500; padding: 10px;'>⚠️ Nhập ít nhất 3 ký tự</div>"
        end_suggestions_container.children = []
        return

    end_suggestions.value = "<div style='color: #4ecdc4; padding: 15px; text-align: center;'>🔍 Đang tìm kiếm gợi ý...</div>"
    end_suggestions_container.children = []

    result = search_address_suggestions(query, limit=6)

    if result['success'] and result['suggestions']:
        end_suggestions_data = result['suggestions']

        end_suggestions.value = f"""
        <div style='color: #4ecdc4; font-weight: 500; margin-bottom: 8px; padding: 0 5px;'>
            ✅ Tìm thấy {len(end_suggestions_data)} gợi ý - Nhấn để chọn:
        </div>
        """

        # Tạo buttons cho gợi ý
        suggestion_buttons = create_suggestion_buttons(end_suggestions_data, False)
        end_suggestions_container.children = tuple(suggestion_buttons) # Assign a tuple of buttons

    else:
        end_suggestions_data = []
        end_suggestions.value = f"<div style='color: #ff6b6b; padding: 10px;'>❌ {result['error']}</div>"
        end_suggestions_container.children = []

# Hàm chọn gợi ý (thông qua button)
def select_start_suggestion(index):
    global selected_start, start_suggestions_data, start_suggestions_container

    if 0 <= index < len(start_suggestions_data):
        selected_start = start_suggestions_data[index]
        start_selected.value = f"""
        <div style='background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%);
                    padding: 12px; border-radius: 8px; margin: 8px 0; color: #333;'>
            <b>✅ Đã chọn điểm đi:</b><br>
            <div style='font-size: 14px; margin-top: 4px;'>{selected_start['display_name']}</div>
        </div>
        """
        # Ẩn gợi ý sau khi chọn
        start_suggestions.value = ""
        start_suggestions_container.children = []

def select_end_suggestion(index):
    global selected_end, end_suggestions_data, end_suggestions_container

    if 0 <= index < len(end_suggestions_data):
        selected_end = end_suggestions_data[index]
        end_selected.value = f"""
        <div style='background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
                    padding: 12px; border-radius: 8px; margin: 8px 0; color: #333;'>
            <b>✅ Đã chọn điểm đến:</b><br>
            <div style='font-size: 14px; margin-top: 4px;'>{selected_end['display_name']}</div>
        </div>
        """
        # Ẩn gợi ý sau khi chọn
        end_suggestions.value = ""
        end_suggestions_container.children = []

# Gán sự kiện
start_search_btn.on_click(search_start_suggestions)
end_search_btn.on_click(search_end_suggestions)

# Tự động tìm kiếm khi gõ (debounced)
last_start_query = ""
last_end_query = ""

def auto_search_start(change):
    global last_start_query
    query = change['new'].strip()
    if query != last_start_query and len(query) >= 3:
        last_start_query = query
        # Delay để tránh spam API
        time.sleep(0.5)
        if query == start_input.value.strip():  # Kiểm tra user vẫn đang gõ cùng query
            search_start_suggestions(None)

def auto_search_end(change):
    global last_end_query
    query = change['new'].strip()
    if query != last_end_query and len(query) >= 3:
        last_end_query = query
        time.sleep(0.5)
        if query == end_input.value.strip():
            search_end_suggestions(None)

start_input.observe(auto_search_start, names='value')
end_input.observe(auto_search_end, names='value')

# Widget tùy chọn
vehicle_selector = widgets.Dropdown(
    options=['🚗 Ô tô', '🏍️ Xe máy'],
    value='🚗 Ô tô',
    description='Loại xe:',
    layout=widgets.Layout(width='200px')
)

weather_selector = widgets.Dropdown(
    options=['☀️ Nắng', '🌧️ Mưa', '🌪️ Bão'],
    value='☀️ Nắng',
    description='Thời tiết:',
    layout=widgets.Layout(width='200px')
)

traffic_selector = widgets.Dropdown(
    options=['🟢 Thấp điểm', '🔴 Cao điểm'],
    value='🟢 Thấp điểm',
    description='Giao thông:',
    layout=widgets.Layout(width='200px')
)

# Nút tính toán
calculate_btn = widgets.Button(
    description='🚀 TÌM ĐƯỜNG & TÍNH PHÍ',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px'),
    style={'font_weight': 'bold'}
)

# Widget hiển thị kết quả
result_display = widgets.HTML()
rating_section = widgets.VBox([])

# Hàm tính toán và hiển thị
def calculate_route(b):
    global selected_start, selected_end

    # Kiểm tra đã chọn điểm chưa
    if not selected_start:
        result_display.value = """
        <div style='background: #ffe6e6; padding: 15px; border-radius: 8px; border-left: 4px solid #ff4757; margin: 10px 0;'>
            <b>⚠️ Chưa chọn điểm đi</b><br>
            Vui lòng tìm kiếm và chọn điểm đi từ gợi ý
        </div>
        """
        return

    if not selected_end:
        result_display.value = """
        <div style='background: #ffe6e6; padding: 15px; border-radius: 8px; border-left: 4px solid #ff4757; margin: 10px 0;'>
            <b>⚠️ Chưa chọn điểm đến</b><br>
            Vui lòng tìm kiếm và chọn điểm đến từ gợi ý
        </div>
        """
        return

    try:
        # Hiển thị loading
        result_display.value = """
        <div style='text-align: center; padding: 20px; color: #4ecdc4;'>
            <h3>🔄 Đang tính toán tuyến đường...</h3>
            <div>Vui lòng chờ một chút</div>
        </div>
        """

        start_coords = selected_start['coordinates']
        end_coords = selected_end['coordinates']

        # Lấy tuyến đường thực tế
        route_coords, distance, duration = get_osrm_route(start_coords, end_coords)

        if route_coords is None:
            route_coords = [start_coords, end_coords]
            distance = geodesic(start_coords, end_coords).meters
            duration = (distance / 1000) / 30 * 3600  # Ước tính 30km/h

        # Tạo bản đồ
        center_lat = (start_coords[0] + end_coords[0]) / 2
        center_lng = (start_coords[1] + end_coords[1]) / 2
        m = folium.Map(location=[center_lat, center_lng], zoom_start=12)

        # Thêm markers
        folium.Marker(
            start_coords,
            popup=f"<b>Điểm đi:</b><br>{selected_start['display_name']}",
            icon=folium.Icon(color='green', icon='play', prefix='fa')
        ).add_to(m)

        folium.Marker(
            end_coords,
            popup=f"<b>Điểm đến:</b><br>{selected_end['display_name']}",
            icon=folium.Icon(color='red', icon='stop', prefix='fa')
        ).add_to(m)

        # Thêm tuyến đường
        folium.PolyLine(
            route_coords,
            color='#4285f4',
            weight=5,
            opacity=0.8,
            popup=f'Tuyến đường: {distance/1000:.2f} km'
        ).add_to(m)

        m.fit_bounds(route_coords)

        # Tính phí
        vehicle = vehicle_selector.value
        weather = weather_selector.value
        traffic = traffic_selector.value

        # Hệ số
        weather_factors = {"☀️ Nắng": 1.0, "🌧️ Mưa": 1.3, "🌪️ Bão": 2.0}
        traffic_factors = {"🟢 Thấp điểm": 1.0, "🔴 Cao điểm": 1.8}

        # Define weather_factor and traffic_factor
        weather_factor = weather_factors[weather]
        traffic_factor = traffic_factors[traffic]


        # Sử dụng hàm tính phí mới
        distance_km = distance / 1000
        calculated_fare = calculate_fare_12k_per_km(distance_km)


        # final_fare = base_fare * weather_factor * traffic_factor

        # if "🏍️" in vehicle:
        #     final_fare *= 0.7

        # Tính thời gian
        final_duration = duration * weather_factor * (traffic_factor * 0.8)

        # Thời gian đến
        booking_time = get_current_datetime()
        arrival_time = booking_time['datetime_obj'] + timedelta(seconds=final_duration)

        # Hiển thị kết quả
        result_display.value = f"""
        <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 15px; margin: 15px 0;'>
            <h3 style='margin: 0 0 15px 0; text-align: center;'>📋 THÔNG TIN CHUYẾN ĐI</h3>

            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; margin-bottom: 15px;'>
                <div style='margin-bottom: 8px;'><b>📍 Từ:</b> {selected_start['short_name']}</div>
                <div><b>🎯 Đến:</b> {selected_end['short_name']}</div>
            </div>

            <div style='display: flex; justify-content: space-between; flex-wrap: wrap; gap: 15px;'>
                <div style='background: rgba(255,255,255,0.1); padding: 12px; border-radius: 8px; flex: 1; min-width: 150px;'>
                    <div style='font-size: 24px; font-weight: bold; color: #ffd700;'>{distance/1000:.2f} km</div>
                    <div style='font-size: 12px; opacity: 0.8;'>Quãng đường</div>
                </div>

                <div style='background: rgba(255,255,255,0.1); padding: 12px; border-radius: 8px; flex: 1; min-width: 150px;'>
                    <div style='font-size: 24px; font-weight: bold; color: #ff6b6b;'>{calculated_fare:,.0f} VND</div>
                    <div style='font-size: 12px; opacity: 0.8;'>Phí cước (12k/km)</div>
                </div>

                <div style='background: rgba(255,255,255,0.1); padding: 12px; border-radius: 8px; flex: 1; min-width: 150px;'>
                    <div style='font-size: 24px; font-weight: bold; color: #4ecdc4;'>{final_duration/60:.0f} phút</div>
                    <div style='font-size: 12px; opacity: 0.8;'>Thời gian di chuyển</div>
                </div>
            </div>

            <div style='background: rgba(255,255,255,0.1); padding: 12px; border-radius: 8px; margin-top: 15px; text-align: center;'>
                <b>🕐 Thời gian đặt:</b> {booking_time['time']} &nbsp;&nbsp;
                <b>⏰ Dự kiến đến nơi:</b> {arrival_time.strftime('%H:%M')}
            </div>

            <div style='margin-top: 15px; font-size: 14px; opacity: 0.9; text-align: center;'>
                {vehicle} • {weather} • {traffic}
            </div>
        </div>
        """

        # Tạo section đánh giá
        rating_slider = widgets.IntSlider(value=5, min=1, max=5, description='⭐ Đánh giá:')
        rating_display = widgets.HTML()
        submit_rating = widgets.Button(description='GỬI ĐÁNH GIÁ', button_style='info')
        rating_result = widgets.HTML()

        def update_rating(change):
            stars = '⭐' * change['new']
            rating_display.value = f"<div style='text-align: center; font-size: 18px;'>{stars}</div>"

        def submit_rating_func(b):
            rating_result.value = """
            <div style='background: #d4edda; color: #155724; padding: 10px; border-radius: 5px; text-align: center; margin-top: 10px;'>
                <b>✅ Cảm ơn bạn đã đánh giá! 🙏</b>
            </div>
            """

        rating_slider.observe(update_rating, names='value')
        submit_rating.on_click(submit_rating_func)
        update_rating({'new': 5})

        global rating_section
        rating_section = widgets.VBox([
            widgets.HTML("<div style='text-align: center; margin: 20px 0;'><h4>💬 Đánh giá chuyến đi</h4></div>"),
            rating_slider,
            rating_display,
            submit_rating,
            rating_result
        ])

        # Hiển thị bản đồ
        display(m)
        display(rating_section)

    except Exception as e:
        result_display.value = f"""
        <div style='background: #ffe6e6; padding: 15px; border-radius: 8px; border-left: 4px solid #ff4757; margin: 10px 0;'>
            <b>❌ Có lỗi xảy ra:</b><br>
            {str(e)}
        </div>
        """

calculate_btn.on_click(calculate_route)

# Hiển thị giao diện
display(header)
display(widgets.HTML("<h4 style='color: #4ecdc4; margin: 15px 0;'>🔍 TÌM KIẾM THÔNG MINH</h4>"))

# Điểm đi
start_box = widgets.VBox([
    widgets.HTML("<div style='font-weight: 500; margin-bottom: 8px; color: #333;'>📍 Điểm đi</div>"),
    widgets.HBox([start_input, start_search_btn], layout=widgets.Layout(align_items='stretch')),
    start_selected,
    start_suggestions,
    start_suggestions_container # Add the container here
])

# Điểm đến
end_box = widgets.VBox([
    widgets.HTML("<div style='font-weight: 500; margin-bottom: 8px; color: #333;'>🎯 Điểm đến</div>"),
    widgets.HBox([end_input, end_search_btn], layout=widgets.Layout(align_items='stretch')),
    end_selected,
    end_suggestions,
    end_suggestions_container # Add the container here
])

display(start_box)
display(widgets.HTML("<br>"))
display(end_box)

# Tùy chọn
display(widgets.HTML("<h4 style='color: #4ecdc4; margin: 20px 0 10px 0;'>⚙️ TÙY CHỌN</h4>"))
display(widgets.HBox([vehicle_selector, weather_selector, traffic_selector]))

# Nút tính toán
display(widgets.HTML("<br>"))
display(widgets.HBox([calculate_btn], layout=widgets.Layout(justify_content='center')))

# Kết quả
display(result_display)

# Hướng dẫn sử dụng
help_info = widgets.HTML(f"""
<div style='background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
            padding: 15px; border-radius: 10px; margin: 20px 0; color: #333;'>
    <h4 style='margin-top: 0; color: #d35400;'>💡 HƯỚNG DẪN SỬ DỤNG</h4>
    <ul style='margin-bottom: 0; line-height: 1.6;'>
        <li><b>Bước 1:</b> Nhập địa chỉ điểm đi (ít nhất 3 ký tự) và chọn từ gợi ý</li>
        <li><b>Bước 2:</b> Nhập địa chỉ điểm đến và chọn từ gợi ý</li>
        <li><b>Bước 3:</b> Chọn loại xe, thời tiết và tình hình giao thông</li>
        <li><b>Bước 4:</b> Nhấn "🚀 TÌM ĐƯỜNG & TÍNH PHÍ" để xem kết quả</li>
        <li><b>Tính năng:</b> Tự động tìm kiếm khi gõ, gợi ý thông minh như Google Maps</li>
    </ul>
</div>
""")

examples_info = widgets.HTML(f"""
<div style='background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%);
            padding: 15px; border-radius: 10px; margin: 10px 0; color: #333;'>
    <h4 style='margin-top: 0; color: #2c3e50;'>📝 VÍ DỤ TÌM KIẾM</h4>
    <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px;'>
        <div>
            <b>🏢 Địa danh nổi tiếng:</b><br>
            <small>• Chợ Bến Thành<br>• Bitexco Tower<br>• Sân bay Tân Sơn Nhất</small>
        </div>
        <div>
            <b>🏠 Địa chỉ cụ thể:</b><br>
            <small>• 123 Nguyễn Huệ Q1<br>• Đại học Bách Khoa<br>• Vincom Đồng Khởi</small>
        </div>
    </div>
</div>
""")

display(help_info)
display(examples_info)

# Footer
footer = widgets.HTML(f"""
<div style='text-align: center; padding: 20px; color: #7f8c8d; border-top: 1px solid #ecf0f1; margin-top: 30px;'>
    <div style='font-size: 16px; margin-bottom: 5px;'>🚖 <b> GOCAR - Xe Của Mọi Nhà</b> 🚖</div>
    <div style='font-size: 12px;'>
        Phiên bản 2.0 • Tìm kiếm thông minh • Gợi ý tự động • Tuyến đường thực tế
    </div>
    <div style='font-size: 12px; margin-top: 8px; color: #95a5a6;'>
        💡 Nhập ít nhất 3 ký tự để bắt đầu tìm kiếm
    </div>
</div>
""")

display(footer)

HTML(value="\n<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);\n            color: w…

HTML(value="<h4 style='color: #4ecdc4; margin: 15px 0;'>🔍 TÌM KIẾM THÔNG MINH</h4>")

HTML(value='<br>')

HTML(value="<h4 style='color: #4ecdc4; margin: 20px 0 10px 0;'>⚙️ TÙY CHỌN</h4>")

HTML(value='<br>')

HTML(value='')

HTML(value='\n<div style=\'background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);\n            padding…

HTML(value="\n<div style='background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%);\n            padding:…

HTML(value="\n<div style='text-align: center; padding: 20px; color: #7f8c8d; border-top: 1px solid #ecf0f1; ma…